# Inferential SirGN for undirected graphs

In [5]:
import pandas as pd
import numpy as np

In [6]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])

In [7]:
x = np.array([1,1,1])

In [12]:
y = np.array([[1,2,4]])

In [17]:
a[1]*y

array([[ 4, 10, 24]])

In [14]:
a*y

array([[ 1,  4, 12],
       [ 4, 10, 24],
       [ 7, 16, 36]])

In [11]:
np.matmul(a,y)

array([17, 38, 59])

In [ ]:
import pandas as pd 
class loader:
    
    def __init__(self):
        self.countID=0
        self.G={}
        self.co={}
        self.revco={}
    
    def nodeID(self,x):
        if x not in self.co:
            #self.co is real id: placement
            self.co[x]=self.countID
            self.countID=self.countID+1
            #self.revco is placement: real id
            self.revco[self.co[x]]=x
        return self.co[x]
    
    def read(self,data):
        #x=pd.read_csv(file,sep=' ',header=None).values
        x = data.values
        for a in range(x.shape[0]):
            i=self.nodeID(x[a,0])
            j=self.nodeID(x[a,1])
            self.addEdge((i,j))
        self.fixG()
        
    def storeEmb(self,file,data):
        file1 = open(file, 'w') 
        for a in range(data.shape[0]):
            s=''+str(int(self.revco[a]))
            for b in range(data.shape[1]):
                s+=' '+str(data[a,b])
            file1.write(s+"\n")
        file1.close()
            
    
    def fixG(self):
        for g in range(len(self.G)):
            self.G[g]=np.array([x for x in self.G[g]])

    def addEdge(self,s):
        (l1,l2)=s
        if l1 not in self.G:
            self.G[l1]=set()
        if l2 not in self.G:
            self.G[l2]=set()
        self.G[l1].add(l2)
        self.G[l2].add(l1)

In [ ]:
class iSirGN:
    
    def __init__(self,n,n1,depth,ngr):
        self.levelsProcessor=[]
        self.n=n
        self.depth=depth
        self.n1=n1
        #self.pca= IncrementalPCA(n_components=n1)
        self.scaler=StandardScaler()
        self.ngr=ngr
        
    
    def transformInternal(self,G):
        nv=len(G) 
        degree=np.array([[G[x].shape[0]] for x in range(nv)])
        emb=np.hstack([degree,np.zeros((nv,n-1))])
        #emb=degree*np.ones((nv,self.n))
        sc=StandardScaler()
        for gmm in self.levelsProcessor:
            emb1=sc.fit_transform(emb)
            val=gmm.transform(emb1)
            M=val.max(axis=1)
            m=val.min(axis=1)
            subx=(M.reshape(nv,1)-val)/(M-m).reshape(nv,1)
            #print(subx.shape)
            su=subx.sum(axis=1)
            #print(su.shape)
            subx=subx/su.reshape(nv,1)
            #print(np.max(np.sum(subx,axis=1)))
            hh=[subx[G[i],:].sum(axis=0) if len(G[i])>0 else np.zeros(self.n) for  i in range(nv)]
            emb=np.vstack(hh)
        #print(emb)
        return emb
    
    def transformGraph(self,G):
        emb1=self.transformInternal(G)
        nv=len(G) 
        val=self.gkm.transform(emb1)
        M=val.max(axis=1)
        m=val.min(axis=1)
        subx=(M.reshape(nv,1)-val)/(M-m).reshape(nv,1)
        #print(subx.shape)
        su=subx.sum(axis=1)
        #print(su.shape)
        subx=subx/su.reshape(nv,1)
        reprr=np.zeros(( self.ngr, self.ngr))
        for  i in range(nv):
            for j in G[i]:
                reprr+=subx[i].reshape((self.ngr,1))*subx[j].reshape((1,self.ngr))
        return reprr
    
    
    
    def transformGraph1(self,G):
        emb1=self.transform2(G)
        nv=len(G) 
        val=self.gkm.predict(emb1)
        reprr=np.zeros(( self.ngr, self.ngr))
        for  i in range(nv):
            for j in G[i]:
                reprr[val[i],val[j]]+=1
        return reprr     
                
    def edge(self,s):
        (a,b)=s
        if a>b:
            return (b,a)
        else:
            return (a,b)
    
    def generateRandomGraph(self):
        if len(self.G)>0:
            if len(self.edges)==0:
                self.edges=set()
                for i in range(len(self.G )):
                    for h in self.G[i]:
                        self.edges.add(self.edge((i,h)))
                self.edges1=np.array([[a,b] for (a,b) in self.edges])
            G1={x:set() for x in self.G}
            edgepercentage=int(self.edges1.shape[0]*(0.1+0.8*np.random.rand()))
            ss=np.random.choice(np.array([i for i in range(self.edges1.shape[0])]), edgepercentage,replace=False)
            for x in self.edges1[ss,:]:
                G1[x[0]].add(x[1])
                G1[x[1]].add(x[0])
            edgepercentage=int(self.edges1.shape[0]*(0.1+0.4*np.random.rand()))
            for i in range(edgepercentage):
                a=random.randint(0,len(self.G )-1)
                b=random.randint(0,len(self.G )-1)
                while b in G1[a]:
                    a=random.randint(0,len(self.G )-1)
                    b=random.randint(0,len(self.G )-1)
                G1[a].add(b)
                G1[b].add(a)   
            G1={x:np.array(list(G1[x])) for x in self.G}
            return G1
        else:
            G1={x:set() for x in range(self.nnodes)}
            edgepercentage=int(self.nnodes*10*(0.1+0.9*np.random.rand()))
            for i in range(edgepercentage):
                a=random.randint(0,self.nnodes-1)
                b=random.randint(0,self.nnodes-1)
                while b in G1[a]:
                    a=random.randint(0,self.nnodes-1)
                    b=random.randint(0,self.nnodes-1)
                G1[a].add(b)
                G1[b].add(a)   
            G1={x:np.array(list(G1[x])) for x in range(self.nnodes)}
            return G1
    
    def fit_graph_representation(self,epochs):
        gmm= MiniBatchKMeans(n_clusters=self.ngr)
        for i in range(epochs):
            #print('epochs',i)
            G=self.generateRandomGraph()
            gmm.partial_fit(self.transformInternal(G))
        return gmm
    
    def fit_gmm(self, epochs):
        gmm= MiniBatchKMeans(n_clusters=self.n)
        for i in range(epochs):
            #print('epochs',i)
            G=self.generateRandomGraph()
            gmm.partial_fit(self.transformInternal(G))
        return (gmm)
        
        
    def universalFit(self,nnodes,epochs):
        self.levelsProcessor=[]
        self.nnodes=nnodes
        for i in range(self.depth):
            print('depth',i)
            self.levelsProcessor.append(self.fit_gmm(epochs))
        #self.fit_scaler_PCA(epochs)
        self.gkm=self.fit_graph_representation(epochs)
        self.edges=set()
        return self       

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score
from sklearn.decomposition import PCA,IncrementalPCA,TruncatedSVD

class iSirGN2:
    
    def __init__(self,n,ngr,depth,levels=10):
        self.n=n
        self.ngr=ngr
        self.leveles=levels
        self.levelProcess=[]
        self.graphRepresentation=(MinMaxScaler(),KMeans(n_clusters=ngr, random_state=1))
        
    def generateRandomGraph(self):
        if len(self.G)>0:
            if len(self.edges)==0:
                self.edges=set()
                for i in range(len(self.G )):
                    for h in self.G[i]:
                        self.edges.add(self.edge((i,h)))
                self.edges1=np.array([[a,b] for (a,b) in self.edges])
            G1={x:set() for x in self.G}
            edgepercentage=int(self.edges1.shape[0]*(0.1+0.8*np.random.rand()))
            ss=np.random.choice(np.array([i for i in range(self.edges1.shape[0])]), edgepercentage,replace=False)
            for x in self.edges1[ss,:]:
                G1[x[0]].add(x[1])
                G1[x[1]].add(x[0])
            edgepercentage=int(self.edges1.shape[0]*(0.1+0.4*np.random.rand()))
            for i in range(edgepercentage):
                a=random.randint(0,len(self.G )-1)
                b=random.randint(0,len(self.G )-1)
                while b in G1[a]:
                    a=random.randint(0,len(self.G )-1)
                    b=random.randint(0,len(self.G )-1)
                G1[a].add(b)
                G1[b].add(a)   
            G1={x:np.array(list(G1[x])) for x in self.G}
            return G1
        else:
            G1={x:set() for x in range(self.nnodes)}
            edgepercentage=int(self.nnodes*10*(0.1+0.9*np.random.rand()))
            for i in range(edgepercentage):
                a=random.randint(0,self.nnodes-1)
                b=random.randint(0,self.nnodes-1)
                while b in G1[a]:
                    a=random.randint(0,self.nnodes-1)
                    b=random.randint(0,self.nnodes-1)
                G1[a].add(b)
                G1[b].add(a)   
            G1={x:np.array(list(G1[x])) for x in range(self.nnodes)}
            return G1
    
    
    def getnumber(self,emb):
        ss=set()
        for x in range(emb.shape[0]):
            sd=''
            for y in range(emb.shape[1]):
                sd+=','+str(emb[x,y])
            ss.add(sd)
        return len(ss)

        
    def fit(self):
        n=self.n
        nv=len(G)
        degree=np.array([[G[x].shape[0]] for x in range(nv)])
        #emb1=degree*np.ones((nv,self.n))
        emb=np.hstack([degree,np.zeros((nv,n-1))])
        for i in range(self.leveles):
            print(i)
            proces=(MinMaxScaler(),MiniBatchKMeans(n_clusters=self.n, random_state=1))
            scaler = proces[0]
            emb1=scaler.fit_transform(emb)
            kmeans = proces[1].fit(emb1)
            val=kmeans.transform(emb1)
            M=val.max(axis=1)
            m=val.min(axis=1)
            subx=(M.reshape(nv,1)-val)/(M-m).reshape(nv,1)
            su=subx.sum(axis=1)
            subx=subx/su.reshape(nv,1)
            hh=[subx[G[i],:].sum(axis=0) if len(G[i])>0 else np.zeros(self.n) for  i in range(nv)]
            emb=np.vstack(hh)
            self.levelProcess.append(proces)

        return emb  

    def transform(self,G,fe):
        n=self.n
        n1=self.n1
        nv=len(G) 
        degree=np.array([[len(G[x]['in'])] for x in range(nv)])
        emb1=np.hstack([degree,np.zeros((nv,n-1))])
        degree=np.array([[len(G[x]['out'])] for x in range(nv)])
        emb2=np.hstack([degree,np.zeros((nv,n-1))])
        emb = np.hstack([emb1, emb2,fe])
        b=[]
        index=0
        dicti={}
        for o in range(nv):
            for j in G[o]['in']:
                f=G[o]['in'][j]
                ##add trasformation features
                b.append(f.reshape((1,f.shape[0])))
                dicti[str(o),"-",str(j)]=index
                index=index+1
        b=np.vstack(b)
        scaler1 = self.edgeFeatures[0]
        embb=scaler1.transform(b)
        kmeans1 = self.edgeFeatures[1]
        val1=kmeans1.transform(embb)
        M1=val1.max(axis=1)
        m1=val1.min(axis=1)
        nv1 = b.shape[0]
        subx1=(M1.reshape(nv1,1)-val1)/(M1-m1).reshape(nv1,1)
        #print(subx.shape)
        su1=subx1.sum(axis=1)
        #print(su.shape)
        subx1=subx1/su1.reshape(nv1,1)
        #print(subx1)
        for i in range(len(self.levelProcess)):
            print(i)
            scaler = self.levelProcess[i][0]
            emb3=scaler.transform(emb)
            kmeans = self.levelProcess[i][1]
            val=kmeans.transform(emb3)
            #print(val)
            M=val.max(axis=1)
            m=val.min(axis=1)
            subx=(M.reshape(nv,1)-val)/(M-m).reshape(nv,1)
            #print(subx.shape)
            su=subx.sum(axis=1)
            #print(su.shape)
            subx=subx/su.reshape(nv,1)
            #print(su.shape)
            hh=np.zeros((nv,n*n1))
            for o in range(nv):
                for j in G[o]['in']:
                    index=dicti[str(o),"-",str(j)]
                    f=G[o]['in'][j]
                    hh[o,:]+= (subx[j,:].reshape((n,1))*subx1[index,:]).flatten()
            emb4=np.vstack(hh)
            hh=np.zeros((nv,n*n1))
            for o in range(nv):
                for j in G[o]['out']:
                    index=dicti[str(j),"-",str(o)]
                    f=G[o]['out'][j]
                    hh[o,:]+= (subx[j,:].reshape((n,1))*subx1[index,:]).flatten()
            emb5=np.vstack(hh)
            emb = np.hstack([emb4, emb5,fe])
        return emb,subx1 
    
    def transformGraph(self,G):
        nv=len(G) 
        emb,sss=self.transform(G)
        graphemb=self.graphRepresentation[0].transform(emb)
        val1=self.graphRepresentation[1].transform(graphemb)
        M1=val1.max(axis=1)
        m1=val1.min(axis=1)
        
        
        subx1=(M1.reshape(nv,1)-val1)/(M1-m1).reshape(nv,1)
        #print(subx.shape)
        su1=subx1.sum(axis=1)
        #print(su.shape)
        subx1=subx1/su1.reshape(nv,1)
        n2= self.n2
        return subx1.sum(axis=0).reshape((1,n2))
    
    def transformGraph1(self,G,fe):
        nv=len(G)
        n=self.n
        n1=self.n1
        n2= self.n2
        emb,sss,dicti=self.transform(G,fe)
        graphemb=self.graphRepresentation[0].transform(emb)
        val1=self.graphRepresentation[1].transform(graphemb)
        M1=val1.max(axis=1)
        m1=val1.min(axis=1)
        subx1=(M1.reshape(nv,1)-val1)/(M1-m1).reshape(nv,1)
        #print(subx.shape)
        su1=subx1.sum(axis=1)
        #print(su.shape)
        subx1=subx1/su1.reshape(nv,1)
        vec=np.zeros(n2*n2*n1)
        for o in range(nv):
                for j in G[o]['in']:
                    index=dicti[str(o),"-",str(j)]
                    vec+=(((subx1[o].reshape((n2,1))*sss[index,:].reshape((1,n1))).flatten().reshape((n2*n1,1)))*subx1[j].reshape((1,n2))).flatten()
        return vec.reshape((1,vec.shape[0]))
     
        def fitUniversal(self, nnodes, epochs):
            self.nnodes = nnodes
            gmm= MiniBatchKMeans(n_clusters=self.n)
            for i in range(epochs):
                G = generateRandomGraph()
                emb = 
            graphemb=self.graphRepresentation[0].fit_transform(emb)
            self.graphRepresentation[1].fit(graphemb)
        